In [2]:
import os
import numpy as np
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms.functional as TF
import random

# Data paths
data_path = r"/kaggle/input/monusegdataset/data"

# ============================================================================
# DATASET PROCESSING
# ============================================================================

# Data Augmentation Class
class NucleiAugmentation:
    def __init__(self, image_size=256, augmentation_prob=0.7, sparse_mode=False):
        self.image_size = image_size
        self.augmentation_prob = augmentation_prob
        self.sparse_mode = sparse_mode
    
    def __call__(self, image, mask):
        if torch.is_tensor(image):
            image = TF.to_pil_image(image)
        if torch.is_tensor(mask):
            mask = TF.to_pil_image(mask)
        
        # Random horizontal flip
        if random.random() > self.augmentation_prob:
            image = TF.hflip(image)
            mask = TF.hflip(mask)
        
        # Random vertical flip
        if random.random() > self.augmentation_prob:
            image = TF.vflip(image)
            mask = TF.vflip(mask)
        
        # Random rotation (-45 to +45 degrees)
        if random.random() > self.augmentation_prob:
            angle = random.uniform(-45, 45)
            image = TF.rotate(image, angle)
            mask = TF.rotate(mask, angle)
        
        # Random brightness/contrast adjustment
        if random.random() > self.augmentation_prob:
            brightness_factor = random.uniform(0.8, 1.2)
            image = TF.adjust_brightness(image, brightness_factor)
        
        if random.random() > self.augmentation_prob:
            contrast_factor = random.uniform(0.8, 1.2)
            image = TF.adjust_contrast(image, contrast_factor)
        
        # Random Gaussian blur
        if random.random() > self.augmentation_prob:
            image = TF.gaussian_blur(image, kernel_size=3)
        
        # For sparse mode: random crop
        if self.sparse_mode and random.random() > 0.8:
            width, height = image.size
            crop_size = random.randint(128, 256)
            i = random.randint(0, height - crop_size)
            j = random.randint(0, width - crop_size)
            image = TF.crop(image, i, j, crop_size, crop_size)
            mask = TF.crop(mask, i, j, crop_size, crop_size)
        
        # Ensure final size
        image = image.resize((self.image_size, self.image_size))
        mask = mask.resize((self.image_size, self.image_size))
        
        return image, mask

# Dataset Class
class KMMSDataset(Dataset):
    def __init__(self, image_paths, mask_paths, image_size=256, augment=False, sparse_mode=False):
        self.image_paths = image_paths
        self.mask_paths = mask_paths
        self.image_size = image_size
        self.augment = augment
        self.sparse_mode = sparse_mode
        self.augmentor = NucleiAugmentation(image_size, sparse_mode=sparse_mode) if augment else None
        
        self.valid_pairs = list(zip(image_paths, mask_paths))
        
        print(f"Created dataset with {len(self.valid_pairs)} valid image-mask pairs")
        if augment:
            mode = "sparse" if sparse_mode else "normal"
            print(f"✓ Data augmentation ENABLED ({mode} mode)")
        else:
            print("✗ Data augmentation DISABLED")
    
    def __len__(self):
        return len(self.valid_pairs)
    
    def __getitem__(self, idx):
        img_path, mask_path = self.valid_pairs[idx]
        
        try:
            # Load image
            image = Image.open(img_path).convert('RGB')
            mask = Image.open(mask_path).convert('L')
            
            # Apply augmentation
            if self.augment and self.augmentor:
                image, mask = self.augmentor(image, mask)
            else:
                image = image.resize((self.image_size, self.image_size))
                mask = mask.resize((self.image_size, self.image_size))
            
            # Convert to numpy arrays
            image = np.array(image).astype(np.float32) / 255.0
            mask = np.array(mask).astype(np.float32) / 255.0
            
            # Ensure mask is binary
            mask = (mask > 0.5).astype(np.float32)
            
            # Convert to tensors
            image = torch.from_numpy(image).permute(2, 0, 1).float()
            mask = torch.from_numpy(mask).unsqueeze(0).float()
            
            return image, mask, os.path.basename(img_path)
        
        except Exception as e:
            print(f"Error loading {img_path}: {e}")
            dummy_image = torch.zeros(3, self.image_size, self.image_size)
            dummy_mask = torch.zeros(1, self.image_size, self.image_size)
            return dummy_image, dummy_mask, "error"

# Data loading and balancing
def find_kmms_data():
    train_path = os.path.join(data_path, "kmms_training")
    test_path = os.path.join(data_path, "kmms_test")
    
    train_images, train_masks, test_images, test_masks = [], [], [], []
    
    # Load training data
    if os.path.exists(train_path):
        train_images_dir = os.path.join(train_path, "images")
        train_masks_dir = os.path.join(train_path, "masks")
        if os.path.exists(train_images_dir):
            train_images = sorted([os.path.join(train_images_dir, f) for f in os.listdir(train_images_dir) 
                                 if f.lower().endswith(('.tiff', '.tif', '.png', '.jpg', '.jpeg'))])
        if os.path.exists(train_masks_dir):
            train_masks = sorted([os.path.join(train_masks_dir, f) for f in os.listdir(train_masks_dir) 
                                if f.lower().endswith(('.tiff', '.tif', '.png', '.jpg', '.jpeg'))])
    
    # Load test data
    if os.path.exists(test_path):
        test_images_dir = os.path.join(test_path, "images")
        test_masks_dir = os.path.join(test_path, "masks")
        if os.path.exists(test_images_dir):
            test_images = sorted([os.path.join(test_images_dir, f) for f in os.listdir(test_images_dir) 
                                if f.lower().endswith(('.tiff', '.tif', '.png', '.jpg', '.jpeg'))])
        if os.path.exists(test_masks_dir):
            test_masks = sorted([os.path.join(test_masks_dir, f) for f in os.listdir(test_masks_dir) 
                               if f.lower().endswith(('.tiff', '.tif', '.png', '.jpg', '.jpeg'))])
    
    print(f"Found: {len(train_images)} training images, {len(test_images)} test images")
    return train_images, train_masks, test_images, test_masks

# Improved data splitting function
def create_balanced_split(all_images, all_masks, test_size=0.15, val_size=0.15, random_state=42):
    """
    Create a balanced train/val/test split from all available data
    """
    from sklearn.model_selection import train_test_split
    
    # First split: Train+Val vs Test
    train_val_images, test_images, train_val_masks, test_masks = train_test_split(
        all_images, all_masks, test_size=test_size, random_state=random_state
    )
    
    # Second split: Train vs Val from train_val set
    val_ratio = val_size / (1 - test_size)
    train_images, val_images, train_masks, val_masks = train_test_split(
        train_val_images, train_val_masks, test_size=val_ratio, random_state=random_state
    )
    
    total_samples = len(all_images)
    print(f"✅ Balanced Data Split:")
    print(f"   Training: {len(train_images)} images ({len(train_images)/total_samples*100:.1f}%)")
    print(f"   Validation: {len(val_images)} images ({len(val_images)/total_samples*100:.1f}%)")
    print(f"   Test: {len(test_images)} images ({len(test_images)/total_samples*100:.1f}%)")
    
    return train_images, val_images, test_images, train_masks, val_masks, test_masks

# Main dataset preparation function
def prepare_datasets(image_size=256, batch_size=4, balanced_split=True):
    """
    Main function to prepare datasets with optional balanced splitting
    """
    # Load all data
    train_images, train_masks, test_images, test_masks = find_kmms_data()
    
    if balanced_split:
        # Combine all data for balanced splitting
        all_images = train_images + test_images
        all_masks = train_masks + test_masks
        
        # Create balanced split (70-15-15 recommended)
        train_images, val_images, test_images, train_masks, val_masks, test_masks = create_balanced_split(
            all_images, all_masks, test_size=0.15, val_size=0.15
        )
    else:
        # Use original split (80-20 from training data for validation)
        from sklearn.model_selection import train_test_split
        train_images, val_images, train_masks, val_masks = train_test_split(
            train_images, train_masks, test_size=0.2, random_state=42
        )
        print(f"Using original split: Train={len(train_images)}, Val={len(val_images)}, Test={len(test_images)}")
    
    # Create datasets
    train_dataset = KMMSDataset(train_images, train_masks, image_size=image_size, augment=True, sparse_mode=True)
    val_dataset = KMMSDataset(val_images, val_masks, image_size=image_size, augment=False, sparse_mode=False)
    test_dataset = KMMSDataset(test_images, test_masks, image_size=image_size, augment=False, sparse_mode=False)
    
    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
    
    return train_loader, val_loader, test_loader, train_dataset, val_dataset, test_dataset

# Example usage
if __name__ == "__main__":
    # Prepare datasets with balanced splitting (recommended)
    train_loader, val_loader, test_loader, train_dataset, val_dataset, test_dataset = prepare_datasets(
        image_size=256, 
        batch_size=4, 
        balanced_split=True  # Set to False to use original split
    )
    
    print(f"\nFinal dataset sizes:")
    print(f"Training: {len(train_dataset)} samples")
    print(f"Validation: {len(val_dataset)} samples") 
    print(f"Test: {len(test_dataset)} samples")
    
    # Test data loading
    print(f"\nTesting data loader...")
    for images, masks, filenames in train_loader:
        print(f"Batch - Images: {images.shape}, Masks: {masks.shape}")
        break

Found: 24 training images, 58 test images
✅ Balanced Data Split:
   Training: 56 images (68.3%)
   Validation: 13 images (15.9%)
   Test: 13 images (15.9%)
Created dataset with 56 valid image-mask pairs
✓ Data augmentation ENABLED (sparse mode)
Created dataset with 13 valid image-mask pairs
✗ Data augmentation DISABLED
Created dataset with 13 valid image-mask pairs
✗ Data augmentation DISABLED

Final dataset sizes:
Training: 56 samples
Validation: 13 samples
Test: 13 samples

Testing data loader...
Batch - Images: torch.Size([4, 3, 256, 256]), Masks: torch.Size([4, 1, 256, 256])


In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# ============================================================================
# FIXED VM-UNET MODEL ARCHITECTURE
# ============================================================================

class VMBlock(nn.Module):
    """Vision Mamba Block with residual connection"""
    def __init__(self, channels):
        super().__init__()
        self.norm = nn.LayerNorm(channels)
        self.conv1 = nn.Conv2d(channels, channels, 3, padding=1)
        self.conv2 = nn.Conv2d(channels, channels, 3, padding=1)
        self.activation = nn.GELU()
        
    def forward(self, x):
        residual = x
        # LayerNorm and channel-first for conv
        x = x.permute(0, 2, 3, 1)  # [B, C, H, W] -> [B, H, W, C]
        x = self.norm(x)
        x = x.permute(0, 3, 1, 2)  # [B, H, W, C] -> [B, C, H, W]
        
        x = self.conv1(x)
        x = self.activation(x)
        x = self.conv2(x)
        
        return x + residual

class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.GELU(),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.GELU()
        )
    
    def forward(self, x):
        return self.double_conv(x)

class VMEncoderBlock(nn.Module):
    """Encoder block with VMBlock"""
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv = DoubleConv(in_channels, out_channels)
        self.vm_block = VMBlock(out_channels)
        self.pool = nn.MaxPool2d(2)
    
    def forward(self, x):
        # Feature extraction
        x = self.conv(x)
        # Mamba-style processing
        x = self.vm_block(x)
        # Downsample
        pooled = self.pool(x)
        return pooled, x

class VMDecoderBlock(nn.Module):
    """FIXED Decoder block with proper channel handling"""
    def __init__(self, in_channels, out_channels):
        super().__init__()
        # Upsample reduces channels by 2
        self.up = nn.ConvTranspose2d(in_channels, in_channels // 2, kernel_size=2, stride=2)
        # After concatenation: (in_channels//2 + skip_channels) -> out_channels
        self.conv = DoubleConv(in_channels // 2 + out_channels, out_channels)  # FIXED
        self.vm_block = VMBlock(out_channels)
    
    def forward(self, x, skip):
        # Upsample
        x = self.up(x)
        # Skip connection - ensure spatial dimensions match
        if x.shape[2:] != skip.shape[2:]:
            x = F.interpolate(x, size=skip.shape[2:], mode='bilinear', align_corners=True)
        
        # Concatenate along channel dimension
        x = torch.cat([x, skip], dim=1)
        # Feature extraction
        x = self.conv(x)
        # Mamba-style processing
        x = self.vm_block(x)
        return x

class VMUNet(nn.Module):
    def __init__(self, n_channels=3, n_classes=1):
        super().__init__()
        
        # Encoder path
        self.enc1 = VMEncoderBlock(n_channels, 64)    # 64 channels
        self.enc2 = VMEncoderBlock(64, 128)           # 128 channels  
        self.enc3 = VMEncoderBlock(128, 256)          # 256 channels
        self.enc4 = VMEncoderBlock(256, 512)          # 512 channels
        
        # Bridge
        self.bridge = DoubleConv(512, 1024)           # 1024 channels
        
        # Decoder path - FIXED channel dimensions
        self.dec1 = VMDecoderBlock(1024, 512)         # 1024->512 after up, then 512+512=1024->512
        self.dec2 = VMDecoderBlock(512, 256)          # 512->256 after up, then 256+256=512->256
        self.dec3 = VMDecoderBlock(256, 128)          # 256->128 after up, then 128+128=256->128
        self.dec4 = VMDecoderBlock(128, 64)           # 128->64 after up, then 64+64=128->64
        
        # Output
        self.outc = nn.Conv2d(64, n_classes, kernel_size=1)
        
    def forward(self, x):
        # Encoder
        x1, skip1 = self.enc1(x)    # skip1: 64 channels
        x2, skip2 = self.enc2(x1)   # skip2: 128 channels
        x3, skip3 = self.enc3(x2)   # skip3: 256 channels
        x4, skip4 = self.enc4(x3)   # skip4: 512 channels
        
        # Bridge
        x5 = self.bridge(x4)        # 1024 channels
        
        # Decoder with proper skip connections
        x = self.dec1(x5, skip4)    # 512 channels
        x = self.dec2(x, skip3)     # 256 channels
        x = self.dec3(x, skip2)     # 128 channels
        x = self.dec4(x, skip1)     # 64 channels
        
        # Output
        return torch.sigmoid(self.outc(x))

# ============================================================================
# SIMPLIFIED ENHANCED VERSION (Easier to debug)
# ============================================================================

class SimpleVMUNet(nn.Module):
    """Simplified version that's easier to debug"""
    def __init__(self, n_channels=3, n_classes=1):
        super().__init__()
        
        # Encoder
        self.enc1 = DoubleConv(n_channels, 64)
        self.pool1 = nn.MaxPool2d(2)
        self.enc2 = DoubleConv(64, 128)
        self.pool2 = nn.MaxPool2d(2)
        self.enc3 = DoubleConv(128, 256)
        self.pool3 = nn.MaxPool2d(2)
        self.enc4 = DoubleConv(256, 512)
        self.pool4 = nn.MaxPool2d(2)
        
        # Bridge with VMBlock
        self.bridge = DoubleConv(512, 1024)
        self.vm_bridge = VMBlock(1024)
        
        # Decoder
        self.up1 = nn.ConvTranspose2d(1024, 512, 2, 2)
        self.dec1 = DoubleConv(1024, 512)  # 512 (up) + 512 (skip)
        self.vm1 = VMBlock(512)
        
        self.up2 = nn.ConvTranspose2d(512, 256, 2, 2)
        self.dec2 = DoubleConv(512, 256)   # 256 (up) + 256 (skip)
        self.vm2 = VMBlock(256)
        
        self.up3 = nn.ConvTranspose2d(256, 128, 2, 2)
        self.dec3 = DoubleConv(256, 128)   # 128 (up) + 128 (skip)
        self.vm3 = VMBlock(128)
        
        self.up4 = nn.ConvTranspose2d(128, 64, 2, 2)
        self.dec4 = DoubleConv(128, 64)    # 64 (up) + 64 (skip)
        self.vm4 = VMBlock(64)
        
        self.outc = nn.Conv2d(64, n_classes, 1)
        
    def forward(self, x):
        # Encoder
        e1 = self.enc1(x)
        e2 = self.enc2(self.pool1(e1))
        e3 = self.enc3(self.pool2(e2))
        e4 = self.enc4(self.pool3(e3))
        
        # Bridge
        b = self.pool4(e4)
        b = self.bridge(b)
        b = self.vm_bridge(b)
        
        # Decoder
        d1 = self.up1(b)
        d1 = torch.cat([d1, e4], dim=1)
        d1 = self.dec1(d1)
        d1 = self.vm1(d1)
        
        d2 = self.up2(d1)
        d2 = torch.cat([d2, e3], dim=1)
        d2 = self.dec2(d2)
        d2 = self.vm2(d2)
        
        d3 = self.up3(d2)
        d3 = torch.cat([d3, e2], dim=1)
        d3 = self.dec3(d3)
        d3 = self.vm3(d3)
        
        d4 = self.up4(d3)
        d4 = torch.cat([d4, e1], dim=1)
        d4 = self.dec4(d4)
        d4 = self.vm4(d4)
        
        return torch.sigmoid(self.outc(d4))

# ============================================================================
# TEST THE MODEL WITH YOUR DATA
# ============================================================================

def test_model_with_data():
    """Test the model with your data loader to verify it works"""
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Testing on device: {device}")
    
    # Load your data
    train_loader, val_loader, test_loader, _, _, _ = prepare_datasets(
        image_size=256, 
        batch_size=4, 
        balanced_split=True
    )
    
    # Test both models
    models = {
        "Fixed VM-UNet": VMUNet(n_channels=3, n_classes=1),
        "Simple VM-UNet": SimpleVMUNet(n_channels=3, n_classes=1)
    }
    
    for name, model in models.items():
        print(f"\n{'='*50}")
        print(f"Testing {name}")
        print(f"{'='*50}")
        
        model.to(device)
        
        # Test with a batch
        try:
            with torch.no_grad():
                for images, masks, _ in train_loader:
                    images = images.to(device)
                    print(f"Input shape: {images.shape}")
                    
                    outputs = model(images)
                    print(f"Output shape: {outputs.shape}")
                    print(f"✅ {name} works correctly!")
                    break
                    
        except Exception as e:
            print(f"❌ Error in {name}: {e}")
    
    return models

# ============================================================================
# UPDATED TRAINING FUNCTION
# ============================================================================

def train_fixed_vm_unet(model, train_loader, val_loader, num_epochs=50, device='cuda'):
    model.to(device)
    
    criterion = CombinedLoss(alpha=0.7, beta=0.3)
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=5, factor=0.5)
    
    train_losses = []
    val_losses = []
    train_f1_scores = []
    val_f1_scores = []
    
    best_val_f1 = 0.0
    best_model_state = None
    
    print("🚀 Starting Fixed VM-UNet Training...")
    
    for epoch in range(num_epochs):
        # Training
        model.train()
        epoch_train_loss = 0.0
        train_preds = []
        train_targets = []
        
        for images, masks, _ in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs} [Train]'):
            images = images.to(device)
            masks = masks.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, masks)
            loss.backward()
            optimizer.step()
            
            epoch_train_loss += loss.item()
            train_preds.append(outputs.detach())
            train_targets.append(masks.detach())
        
        # Validation
        model.eval()
        epoch_val_loss = 0.0
        val_preds = []
        val_targets = []
        
        with torch.no_grad():
            for images, masks, _ in tqdm(val_loader, desc=f'Epoch {epoch+1}/{num_epochs} [Val]'):
                images = images.to(device)
                masks = masks.to(device)
                
                outputs = model(images)
                loss = criterion(outputs, masks)
                epoch_val_loss += loss.item()
                
                val_preds.append(outputs)
                val_targets.append(masks)
        
        # Calculate metrics
        avg_train_loss = epoch_train_loss / len(train_loader)
        avg_val_loss = epoch_val_loss / len(val_loader)
        
        train_preds = torch.cat(train_preds)
        train_targets = torch.cat(train_targets)
        train_precision, train_recall, train_accuracy, train_f1 = calculate_metrics(train_preds, train_targets, threshold=0.3)
        
        val_preds = torch.cat(val_preds)
        val_targets = torch.cat(val_targets)
        val_precision, val_recall, val_accuracy, val_f1 = calculate_metrics(val_preds, val_targets, threshold=0.3)
        
        # Store results
        train_losses.append(avg_train_loss)
        val_losses.append(avg_val_loss)
        train_f1_scores.append(train_f1)
        val_f1_scores.append(val_f1)
        
        scheduler.step(avg_val_loss)
        
        print(f'\n📈 Epoch {epoch+1}/{num_epochs}:')
        print(f'   Train Loss: {avg_train_loss:.4f} | Prec: {train_precision:.4f} | Rec: {train_recall:.4f} | F1: {train_f1:.4f}')
        print(f'   Val Loss: {avg_val_loss:.4f} | Prec: {val_precision:.4f} | Rec: {val_recall:.4f} | F1: {val_f1:.4f}')
        
        # Save best model
        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
            best_model_state = model.state_dict().copy()
            torch.save(best_model_state, 'best_fixed_vm_unet_model.pth')
            print(f'💾 New best model saved! Val F1: {best_val_f1:.4f}')
    
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
    return model, train_losses, val_losses, train_f1_scores, val_f1_scores

# ============================================================================
# MAIN EXECUTION
# ============================================================================

def main():
    # First test the models
    models = test_model_with_data()
    
    # Use the simple model for training (more stable)
    model = models["Simple VM-UNet"]
    
    # Load datasets
    train_loader, val_loader, test_loader, train_dataset, val_dataset, test_dataset = prepare_datasets(
        image_size=256, 
        batch_size=4, 
        balanced_split=True
    )
    
    print(f"\n📊 Dataset sizes: Train={len(train_dataset)}, Val={len(val_dataset)}, Test={len(test_dataset)}")
    
    # Train the model
    print("\n🎯 Starting training...")
    model, train_losses, val_losses, train_f1_scores, val_f1_scores = train_fixed_vm_unet(
        model, train_loader, val_loader, num_epochs=50, device='cuda'
    )
    
    # Test the model
    test_metrics = test_vm_unet(model, test_loader, device='cuda')
    
    # Save final model
    torch.save(model.state_dict(), 'final_fixed_vm_unet_model.pth')
    print("\n💾 Final model saved!")
    
    return model, test_metrics

if __name__ == "__main__":
    model, metrics = main()

Testing on device: cuda
Found: 24 training images, 58 test images
✅ Balanced Data Split:
   Training: 56 images (68.3%)
   Validation: 13 images (15.9%)
   Test: 13 images (15.9%)
Created dataset with 56 valid image-mask pairs
✓ Data augmentation ENABLED (sparse mode)
Created dataset with 13 valid image-mask pairs
✗ Data augmentation DISABLED
Created dataset with 13 valid image-mask pairs
✗ Data augmentation DISABLED

Testing Fixed VM-UNet
Input shape: torch.Size([4, 3, 256, 256])
Output shape: torch.Size([4, 1, 256, 256])
✅ Fixed VM-UNet works correctly!

Testing Simple VM-UNet
Input shape: torch.Size([4, 3, 256, 256])
Output shape: torch.Size([4, 1, 256, 256])
✅ Simple VM-UNet works correctly!
Found: 24 training images, 58 test images
✅ Balanced Data Split:
   Training: 56 images (68.3%)
   Validation: 13 images (15.9%)
   Test: 13 images (15.9%)
Created dataset with 56 valid image-mask pairs
✓ Data augmentation ENABLED (sparse mode)
Created dataset with 13 valid image-mask pairs
✗ D

Epoch 1/50 [Val]: 100%|██████████| 4/4 [00:00<00:00,  5.23it/s]



📈 Epoch 1/50:
   Train Loss: 0.4892 | Prec: 0.1542 | Rec: 0.9865 | F1: 0.2667
   Val Loss: 0.5331 | Prec: 0.0911 | Rec: 0.0056 | F1: 0.0105
💾 New best model saved! Val F1: 0.0105


Epoch 2/50 [Val]: 100%|██████████| 4/4 [00:00<00:00,  5.06it/s]



📈 Epoch 2/50:
   Train Loss: 0.3721 | Prec: 0.3575 | Rec: 0.8770 | F1: 0.5079
   Val Loss: 0.6211 | Prec: 0.8502 | Rec: 0.1170 | F1: 0.2058
💾 New best model saved! Val F1: 0.2058


...


📈 Epoch 49/50:
   Train Loss: 0.2053 | Prec: 0.7157 | Rec: 0.8316 | F1: 0.7693
   Val Loss: 0.2391 | Prec: 0.6957 | Rec: 0.7867 | F1: 0.7384


Epoch 50/50 [Val]: 100%|██████████| 4/4 [00:00<00:00,  5.09it/s]



📈 Epoch 50/50:
   Train Loss: 0.1815 | Prec: 0.7244 | Rec: 0.8650 | F1: 0.7885
   Val Loss: 0.2356 | Prec: 0.6936 | Rec: 0.7974 | F1: 0.7419

🧪 Testing VM-UNet on test set...


Testing: 100%|██████████| 4/4 [00:00<00:00,  4.78it/s]



🔍 Finding optimal threshold...
   Threshold 0.3: Precision=0.7251, Recall=0.7229, F1=0.7240
   Threshold 0.4: Precision=0.7513, Recall=0.6867, F1=0.7175
   Threshold 0.5: Precision=0.7751, Recall=0.6499, F1=0.7070
   Threshold 0.6: Precision=0.7977, Recall=0.6106, F1=0.6917
   Threshold 0.7: Precision=0.8210, Recall=0.5648, F1=0.6692

🎯 VM-UNet FINAL TEST RESULTS
Optimal Threshold: 0.3
Test Loss: 0.2495
Precision: 0.7251
Recall:    0.7229
Accuracy:  0.9037
F1-Score:  0.7240

💾 Final model saved!
